AI 804 Capstone Project - Group 2

In [1]:
# author: Savitha Kolar
# spring 2024

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag

In [2]:
!pip install spacy


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 12.8/12.8 MB 6.2 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install -q efficientnet


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install --upgrade numpy


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# !pip uninstall tensorflow

In [7]:
!pip install tensorflow


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install googletrans==3.1.0a0

  Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl (53 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.4
    Uninstalling httpcore-1.0.4:
      Successfully uninstalled httpcore-1.0.4
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.21.0 requires httpx>=0.24.1, but you have httpx 0.13.3 which is incompatible.
gradio-client 0.12.0 requires httpx>=0.24.1, but you have httpx 0.13.3 which is incompatible.
openai 1.9.0 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install langid


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
!pip install opencv-python-headless


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import numpy as np 
import pandas as pd
import random
import math
import re
import os
import langid
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.io import imread as imread
from skimage.util import montage
from skimage.color import label2rgb
import cv2
from sklearn.model_selection import train_test_split
import googletrans
from googletrans import Translator
translator = Translator()
import ast
import json
import spacy

# Load the English language model in spacy
nlp = spacy.load('en_core_web_sm')

# Define batch size
batch_size = 100

In [12]:
# import warnings
# import pandas as pd

# # Turn off the SettingWithCopyWarning
# warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

In [13]:
import tensorflow as tf
# import tensorflow.keras.applications as tfka
# import efficientnet.tfkeras as efn

### Functions

In [14]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

# Function to preprocess the recipe and retain only nouns
def preprocess_recipe(recipe):
    doc = nlp(recipe)
    filtered_tokens = []
    for token in doc:
        # Exclude tokens identified as punctuation, stop words, and measurement units
        if token.pos_ == 'NOUN' and token.text.lower() not in {'cup', 'cups', 'tablespoon', 'tablespoons', 'teaspoon', 'teaspoons', 'ounce', 'ounces', 'inch'
                                                               , 'inches', 'pound', 'pounds', 'slice', 'slices', 'white', 'fresh', 'grated', 'leaves', 'cut'
                                                               , 'thinly', 'ground', 'medium', 'plus', 'serving', 'chopped', 'large', 'finely',
                                                              'room temperature', 'lb', 'tsp', 'tsp.', 'tbsp', 'tbsp.', 'oz', 'Pinch', 'diameter'
                                                              , 'pieces', 'powder'} and '/' not in token.text:
            filtered_tokens.append(token.text)
    return ' '.join(filtered_tokens)

# Function to plot word frequencies
def plot_word_frequencies(df, column_name):
    # Flatten the list of words from the specified column
    all_words = ' '.join(df[column_name]).split()
    
    # Create a Pandas Series to count the frequency of each word
    word_freq = pd.Series(all_words).value_counts().sort_values(ascending=False)
    
    # Plot the top 20 most frequent words
    plt.figure(figsize=(10, 6))
    word_freq.head(20).plot(kind='bar')
    plt.title('Top 20 Most Frequent Words')
    plt.xlabel('Words')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45)
    plt.show()
    
# Google translate
def to_translate(original_text):
    langs = translator.detect(original_text)
    print(langs.lang, langs.confidence)
    new_titlename = original_text
    if langs.confidence>0.6:
        totranslate = translator.translate(original_text, dest='en')
        sourcelang = googletrans.LANGUAGES.get(totranslate.src, totranslate.src)
        print(f'Converting from {sourcelang} to english.')
        new_titlename = totranslate.text
    return new_titlename

# Define a function to check for non-English alphabet characters
# To make the pattern more inclusive of various characters commonly used in English, using Unicode character properties
# Tried r'[^\x00-\x7F]' first; wasn't recognizing accents, etc.\u0000-\u007F. Trying Latin-1 Supplement: \x00-\xFF
# Basic Latin Unice: P{IsBasicLatin}
def contains_english(text):    
#     return bool(re.search(r'[^A-Za-z\sÀ-ÿ]', text))
    b=langid.classify(text)[0] == 'en'
    return b

# Define a function to check for english alphabet characters
def is_english_word(word):
    for char in word:
        if not unicodedata.category(char).startswith('L'):
            return False
    return True

In [15]:
seed=42
seed_everything(seed)

In [16]:
def image_mapping_check(dataset):
    counter = 0
    record = []

    while counter < dataset.shape[0]-1:
        row = dataset.loc[counter]
        print(row, counter)
        img_name = row['Image_Name']
        img = cv2.imread(img_path+img_name+'.jpg')
        try:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) 
        except:
            record.append(counter)
        counter+= 1
    print(record.shape)
#     new = dataset.drop(record, axis = 0)
#     new = new.reset_index(drop = True)
#     return new

def allergens_mapping(row, types):
    for item in common_allergens[types]:
        if item.lower() in row.lower():
                return 1
    return 0

def combination(row):
    return(row['milk'], row['meat'], row['eggs'], row['tree nuts'], row['peanuts'], row['shellfish'], row['wheat'], row['soy'], row['fish'])

def vegan(row):
     return (
        not row['milk'],
        not row['meat'],
        not row['eggs'],
        not row['shellfish'],
        not row['fish']
    )

def vegetarian(row):
     return (
        not row['meat'],
        not row['eggs'],
        not row['shellfish'],
        not row['fish']
    )
    
def meat(row):
    return (
        row['meat']
    )

def pescatarian(row):
    return (
        not row['meat'],
        not row['shellfish']
    )

def load_allergen_data(df):
    df = image_mapping_check(df)
    df['milk'] = df[df['Type']=='Cleaned Ingredients']['Recipe'].apply(lambda x: allergens_mapping(x, types = 'milk'))
    df['meat'] = df[df['Type']=='Cleaned Ingredients']['Recipe'].apply(lambda x: allergens_mapping(x, types = 'meat'))    
    df['eggs'] = df[df['Type']=='Cleaned Ingredients']['Recipe'].apply(lambda x: allergens_mapping(x, types = 'eggs'))
    df['tree nuts'] = df[df['Type']=='Cleaned Ingredients']['Recipe'].apply(lambda x: allergens_mapping(x, types = 'tree nuts'))
    df['peanuts'] = df[df['Type']=='Cleaned Ingredients']['Recipe'].apply(lambda x: allergens_mapping(x, types = 'peanuts'))
    df['shellfish'] = df[df['Type']=='Cleaned Ingredients']['Recipe'].apply(lambda x: allergens_mapping(x, types = 'shellfish'))
    df['wheat'] = df[df['Type']=='Cleaned Ingredients']['Recipe'].apply(lambda x: allergens_mapping(x, types = 'wheat'))
    df['soy'] = df[df['Type']=='Cleaned Ingredients']['Recipe'].apply(lambda x: allergens_mapping(x, types = 'soy'))
    df['fish'] = df[df['Type']=='Cleaned Ingredients']['Recipe'].apply(lambda x: allergens_mapping(x, types = 'fish'))
    df['total'] = df.apply(combination, axis = 1)
    df['vegan'] = df.apply(vegan, axis=1)
    df['vegetarian'] = df.apply(vegetarian, axis=1)   
    df['meat'] = df.apply(meat, axis=1)
    df['pescatarian'] = df.apply(pescatarian, axis=1)  

#     x_train, x_val, y_train, y_val = train_test_split(df[['Image_Name','image_path']], df.iloc[:,7:17], shuffle = True, random_state = seed, test_size = 0.25)
#     train_df = pd.concat([x_train, y_train], axis = 1).reset_index(drop = True)
#     val_df = pd.concat([x_val, y_val], axis = 1).reset_index(drop = True)
#     train_df.head()
#     return df, train_df, val_df

def create_train_test_split_old(df, test_size=0.25, random_state=None):
    # Group the DataFrame by 'Title', to ensure all rows eith same Title are in the same dataset.
    grouped = df.groupby('Title')
    train_dfs = []
    val_dfs = []

    for title, group_df in grouped:
        train_group, val_group = train_test_split(group_df, test_size=test_size, random_state=random_state)
#         x_train, x_val, y_train, y_val = train_test_split(df[['Title', 'Recipe', 'image_path']], df.iloc[:,7:17], shuffle = True, random_state = seed, test_size = 0.25)
        train_dfs.append(train_group)
        val_dfs.append(val_group)
        
    # Concatenate train and val sets for each group
    train_df = pd.concat(train_dfs, axis = 0).reset_index(drop = True)
    val_df = pd.concat(val_dfs, axis = 0).reset_index(drop = True)
    
    return df, train_df, val_df

def create_train_test_split(df, features_columns, label_columns, test_size=0.25, random_state=None):
    # Split the DataFrame into features (X) and labels (y)
    X = df[features_columns]
    y = df[label_columns]
    x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=test_size, shuffle = True, random_state = random_state)
    
    train_df = pd.concat([x_train, y_train], axis = 1).reset_index(drop = True)
    val_df = pd.concat([x_val, y_val], axis = 1).reset_index(drop = True)
    train_df.head()
    return df, train_df, val_df


In [17]:
# Get the current directory (where the python file 'p.py' is located)
current_directory = os.path.dirname("Prompts from sentences.ipynb")

print(current_directory)

# Define the folder path where the image files are located
img_path = 'Food Recipes\\Food Images'

In [18]:

# Construct the path to the CSV file located inside the 'f' folder
csv_file_path = os.path.join(current_directory, 'Food Recipes', 'Food Ingredients and Recipe Dataset with Image Name Mapping.csv')

# Read the CSV file into a DataFrame
foodrecipes_df = pd.read_csv(csv_file_path, delimiter=',', encoding="utf-8", keep_default_na=False)

foodrecipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13501 entries, 0 to 13500
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           13501 non-null  int64 
 1   Title                13501 non-null  object
 2   Ingredients          13501 non-null  object
 3   Instructions         13501 non-null  object
 4   Image_Name           13501 non-null  object
 5   Cleaned_Ingredients  13501 non-null  object
dtypes: int64(1), object(5)
memory usage: 633.0+ KB


In [19]:
foodrecipes_df.shape

(13501, 6)

In [20]:
null_rows = foodrecipes_df[foodrecipes_df.isnull().any(axis=1)]
print(null_rows)
# 8 records found without any instructions/process. These will have to be deleted.

Empty DataFrame
Columns: [Unnamed: 0, Title, Ingredients, Instructions, Image_Name, Cleaned_Ingredients]
Index: []


In [21]:
foodrecipes_df = foodrecipes_df.dropna()

In [22]:
foodrecipes_df.shape
# Total records wll be 13501-8 = 13493.

(13501, 6)

In [23]:
# Rename the first column
new_column_names = {'Unnamed: 0': 'Idx'}
foodrecipes_df = foodrecipes_df.rename(columns=new_column_names)

foodrecipes_df.columns

Index(['Idx', 'Title', 'Ingredients', 'Instructions', 'Image_Name',
       'Cleaned_Ingredients'],
      dtype='object')

In [24]:
foodrecipes_df = foodrecipes_df.set_index('Idx')

In [25]:
foodrecipes_df.head()

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
Idx,,,,,
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."


In [26]:
foodrecipes_df.iloc[41]

Title                                                  Iced Café de Olla
Ingredients            ['¾ cup grated piloncillo or (packed) dark bro...
Instructions           Bring piloncillo, canela, cardamom, allspice, ...
Image_Name                                             iced-cafe-de-olla
Cleaned_Ingredients    ['¾ cup grated piloncillo or (packed) dark bro...
Name: 41, dtype: object

In [27]:
# Since we are planning to rename the images with the title names, we want to ensure that the title names are unique, and there are no duplicates.
# Count duplicates based on the 'Title' column
duplicate_counts = foodrecipes_df['Title'].value_counts()

print("Duplicate Counts based on Title:")
print(duplicate_counts)

Duplicate Counts based on Title:
French 75                                                              5
                                                                       5
Potato Latkes                                                          5
Chopped Salad                                                          4
Pickled Red Onions                                                     4
                                                                      ..
Cabbage and Jicama Slaw                                                1
Za'atar Roast Chicken with Green Tahini Sauce                          1
Grilled Corn with Miso Butter                                          1
Grilled Lettuces with Crème Fraîche and Avocado                        1
Mexican Poblano, Spinach, and Black Bean "Lasagne" with Goat Cheese    1
Name: Title, Length: 13306, dtype: int64


There seem to be several duplicate title names for the recipes. For example, there are 5 dishes, all with the same title of 'Potato Latkes'. We will have to rename them. One strategy is to append their row index to the title name so it is unique to the recipe. Error handling will also need to be done so the program either skips the file or handles the exception. One of the exceptions was because of the presence of double quotes in the Title. For the Title and image name to be consistent, if we removed double quotes for the image name, we would have to make sure the Title also had the double quotes removed.

In [28]:
#Print one of the rows for the Potato Ltke recipe
print('Original Row', )
print(foodrecipes_df.iloc[6062], '\n')

# Identify duplicate titles
duplicate_titles = foodrecipes_df.duplicated(subset='Title', keep=False)

# Apply the concatenation only for duplicate titles
foodrecipes_df['Cleaned_title'] = foodrecipes_df.apply(lambda row: f"{row['Title']}_{row.name}" if duplicate_titles[row.name] else row['Title'], axis=1)

# Set the maximum column width to None to display entire content
pd.set_option('display.max_colwidth', None)
print('Updated Row', )
foodrecipes_df.iloc[6062]

Original Row
Title                                                      Potato Latkes
Ingredients            ['1 pound potatoes', '1/2 cup finely chopped o...
Instructions           Preheat oven to 250°F.\nPeel potatoes and coar...
Image_Name                                          potato-latkes-104406
Cleaned_Ingredients    ['1 pound potatoes', '1/2 cup finely chopped o...
Name: 6062, dtype: object 

Updated Row


Title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                Potato Latkes
Ingre

In [30]:
# Statistical summaries
#All the columns are strings - statistical sumaries do not make sense.
foodrecipes_df.describe(include='all')

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Cleaned_title
count,13501,13501,13501,13501,13501,13501
unique,13306,13473,13465,13472,13473,13501
top,French 75,[],,#NAME?,[''],Miso-Butter Roast Chicken With Acorn Squash Panzanella
freq,5,12,8,30,12,1


In [31]:
foodrecipes_df.loc[foodrecipes_df['Image_Name']=='#NAME?'].head()
#  There are 30 records that have an incorrect Image_Name as #NAME? Upon a closer look in the original csv file, all the image names for these files start with a hyphen.
#  We will need to strip the hyphen from the Image_Name and replace with a blank.
# We will DELETE THESE!!

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Cleaned_title
Idx,,,,,,
3441,"""Burnt"" Carrots and Parsnips","['1 1/2 pounds carrots, peeled, halved lengthwise, cut into 4"" pieces', '1 1/2 pounds parsnips, peeled, cut into 4"" pieces', '2 tablespoons olive oil', 'Kosher salt, freshly ground pepper', '1/4 medium onion, finely chopped', '4 tablespoons unsalted butter, cut into pieces', '1/4 cup bourbon', '3 tablespoons dark brown sugar']","Preheat oven to 450°F. Toss carrots, parsnips, and oil on a rimmed baking sheet; season with salt and pepper. Roast, tossing occasionally, until tender and charred in spots, 20–25 minutes.\nRemove vegetables from oven, add onion, butter, bourbon, and brown sugar, and toss to coat. Roast, tossing occasionally, until sugars have caramelized and vegetables are completely softened, 8–10 minutes.\nTransfer vegetables to a platter and pour any juices over.",#NAME?,"['1 1/2 pounds carrots, peeled, halved lengthwise, cut into 4"" pieces', '1 1/2 pounds parsnips, peeled, cut into 4"" pieces', '2 tablespoons olive oil', 'Kosher salt, freshly ground pepper', '1/4 medium onion, finely chopped', '4 tablespoons unsalted butter, cut into pieces', '1/4 cup bourbon', '3 tablespoons dark brown sugar']","""Burnt"" Carrots and Parsnips"
3641,"""Bloody Mary"" Tomato Toast with Celery and Horseradish","['1 lemon, zested, juiced', '1/2 shallot, finely chopped', '1 pound grape tomatoes, halved', '2 tablespoons extra-virgin olive oil', '1 tablespoon Sherry vinegar', '1 teaspoon kosher salt', '1 teaspoon freshly ground black pepper', '1 teaspoon Worcestershire sauce', '1/4 teaspoon cayenne pepper', '5 tablespoons finely grated fresh horseradish, divided', '2 celery stalks, thinly sliced', '3 tablespoons coarsely chopped celery leaves', '6 tablespoons mayonnaise', '6 slices toasted rye bread']","Combine lemon zest, lemon juice, shallot, tomatoes, oil, vinegar, salt, black pepper, Worcestershire sauce, cayenne, and 1 Tbsp. horseradish in a large bowl. Let marinate 10 minutes, then stir in celery stalks and leaves.\nMeanwhile, combine mayonnaise and remaining 4 Tbsp. horseradish in a small bowl. Spread 1 Tbsp. horseradish-mayonnaise on each slice of rye. Top toasts evenly with tomato mixture.",#NAME?,"['1 lemon, zested, juiced', '1/2 shallot, finely chopped', '1 pound grape tomatoes, halved', '2 tablespoons extra-virgin olive oil', '1 tablespoon Sherry vinegar', '1 teaspoon kosher salt', '1 teaspoon freshly ground black pepper', '1 teaspoon Worcestershire sauce', '1/4 teaspoon cayenne pepper', '5 tablespoons finely grated fresh horseradish, divided', '2 celery stalks, thinly sliced', '3 tablespoons coarsely chopped celery leaves', '6 tablespoons mayonnaise', '6 slices toasted rye bread']","""Bloody Mary"" Tomato Toast with Celery and Horseradish"
4117,Smoked Salmon Breakfast Salad with Crispbread,"['2 baby beets or radishes, thinly sliced', '8 thin slices red onion (optional)', '6 cups mesclun', '2 tablespoons fresh lemon juice', '2 tablespoons olive oil', '1 tablespoon chopped capers', 'Kosher salt, freshly ground pepper', '0 freshly ground pepper', '3 ounces smoked salmon or gravlax', '3/4 cup Neufchâtel or cream cheese', '4 Wasa crispbread or toasted pumpernickel slices', '1 lemon, cut into wedges']","Toss beets, onion (if using), mesclun, lemon juice, oil, and capers in a large bowl; season with salt and pepper. Add salmon and gently toss to combine. Mound salad alongside crispbread spread with Neufchâtel and serve with lemon wedges for squeezing over.",#NAME?,"['2 baby beets or radishes, thinly sliced', '8 thin slices red onion (optional)', '6 cups mesclun', '2 tablespoons fresh lemon juice', '2 tablespoons olive oil', '1 tablespoon chopped capers', 'Kosher salt, freshly ground pepper', '0 freshly ground pepper', '3 ounces smoked salmon or gravlax', '3/4 cup Neufchâtel or cream cheese', '4 Wasa crispbread or toasted pumpernickel slices', '1 lemon', 'cut into wedges']",Smoked Salmon Breakfast Salad wi

In [32]:
# Drop all the rows that have #NAME? Image_name, since they are actual images, but are coming thru incorrectly. 
foodrecipes_df = foodrecipes_df.drop(foodrecipes_df[foodrecipes_df['Image_Name']=='#NAME?'].index)

In [33]:
# if '#NAME?' in foodrecipes_df['Image_Name'].values:
#     # Select rows where 'Image_Name' is '#NAME?'
#     rows_to_rename = foodrecipes_df.loc[foodrecipes_df['Image_Name'] == '#NAME?']
#     # Rename the 'Image_Name' column for selected rows with the 'Title' value
#     foodrecipes_df.loc[rows_to_rename.index, 'Image_Name'] = foodrecipes_df.loc[rows_to_rename.index, 'Title']
# foodrecipes_df.iloc[3441]

In [34]:
foodrecipes_df.shape

(13471, 6)

In [35]:
foodrecipes_df.iloc[74]
t=foodrecipes_df.iloc[74]['Title']
langid.classify(t)
t1='Plátanos Horneados con Crema y Queso (Baked Banana With Crema and Cheese)'
if langid.classify(t)[0] != 'en':
    print(t)

Doenjang Jjigae (된장찌개 / Fermented-Soybean Stew)


In [36]:
#  Identify and drop records that have non-english script in the titlename. 
#  Limitation of our app: Currently we will only handle recipes in English.
english_mask = foodrecipes_df['Title'].apply(contains_english)

# Filter English rows; create new df to store cleaned rows
foodrecipescleaned_df = foodrecipes_df[english_mask]

print(foodrecipescleaned_df['Title'])

Idx
0                     Miso-Butter Roast Chicken With Acorn Squash Panzanella
1                                            Crispy Salt and Pepper Potatoes
2                                                Thanksgiving Mac and Cheese
3                                         Italian Sausage and Bread Stuffing
4                                                               Newton's Law
                                        ...                                 
13495                               Dandelion Salad with Warm Bacon Dressing
13497     Israeli Couscous with Roasted Butternut Squash and Preserved Lemon
13498                    Rice with Soy-Glazed Bonito Flakes and Sesame Seeds
13499                                                            Spanakopita
13500    Mexican Poblano, Spinach, and Black Bean "Lasagne" with Goat Cheese
Name: Title, Length: 10344, dtype: object


In [37]:
foodrecipescleaned_df.shape
# After dropping rows that did not conform to our standard of english-script-only recipes, left with 10,344 records.

(10344, 6)

In [38]:
d = foodrecipescleaned_df.copy()
d['Cleaned_Ingredients'] = d['Cleaned_Ingredients'].apply(eval)
d.head()

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Cleaned_title
Idx,,,,,,
0,Miso-Butter Roast Chicken With Acorn Squash Panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more', '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage', '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature', '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper', '⅓ loaf good-quality sturdy white bread, torn into 1"" pieces (about 2½ cups)', '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1"" pieces', '2 Tbsp. extra-virgin olive oil', '½ small red onion, thinly sliced', '3 Tbsp. apple cider vinegar', '1 Tbsp. white miso', '¼ cup all-purpose flour', '2 Tbsp. unsalted butter, room temperature', '¼ cup dry white wine', '2 cups unsalted chicken broth', '2 tsp. white miso', 'Kosher salt, freshly ground pepper']","Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.\nMeanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½""-thick wedges; arrange on a rimmed baking sheet.\nCombine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.\nAdd bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.\nPlace onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.\nPlace a rack in middle and lower third of oven; preheat to 425°F. Mix miso and 3 Tbsp. room-temperature butter in a small bowl until smooth. Pat chicken dry with paper towels, then rub or brush all over with miso butter. Place chicken in a large cast-iron skillet and roast on middle rack until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. (Temperature will climb to 165°F while chicken rests.) Let chicken rest in skillet at least 5 minutes, then transfer to a plate; reserve skillet.\nMeanwhile, roast squash on lower rack until mostly tender, about 25 minutes. Remove from oven and scatter reserved bread mixture over, spreading into as even a layer as you can manage. Return to oven and roast until bread is golden brown and crisp and apples are tender, about 15 minutes. Remove from oven, drain pickled onions, and toss to combine. Transfer to a serving dish.\nUsing your fingers, mash flour and butter in a small bowl to combine.\nSet reserved skillet with chicken drippings over medium heat. You should have about ¼ cup, but a little over or under is all good. (If you have significantly more, drain off and set excess aside.) Add wine and cook, stirring often and scraping up any browned bits with a wooden spoon, until bits are loosened and wine is reduced by about half (you should be able to smell the wine), about 2 minutes. Add butter mixture; cook, stirring often, until a smooth paste forms, about 2 minutes. Add broth and any reserved drippings and cook, stirring constantly, until combined and thickened, 6–8 minutes. Remove from heat and stir in miso. Taste and season with salt and black pepper.\nServe chicken with gravy and squash panzanella alongside.",miso-butter-roast-chicken-acorn-squash-panzanella,"[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher salt, divided, plus more, 2 small acorn squash (about 3 lb. total), 2 Tbsp. finely chopped sage, 1 Tbsp. finely chopped rosemary, 6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature, ¼ tsp. ground allspice, Pinch of crushed red pepper flakes, Freshly ground black pepper, ⅓ loaf good-quality sturdy white bread, torn into 1"" pieces (about 2½ cups), 2 medium a

## Rename the image name with the Title of the recipe. 

### This will be especially useful when we generate a single image using just the Title name, and can compare it with the existing image.

In [40]:
foodrecipescleaned_df.shape

(10344, 6)

In [42]:
#  UNCOMMENT THIS SECTION WHEN YOU DOWNLOAD THE IMAGES FRESH - TO BE RUN ONLY THE FIRST & ONLY TIME!!

# Rename the image file names to match the title names..

import traceback
import pandas as pd

# Define an empty dataframe to store all batch dataframes
foodrecipescleaned_2_df = pd.DataFrame()

# Define batch size
batch_size = 100
aeerror_N = 0
oserror_N = 0
feerror_N = 0
fnferror_N = 0

# Iterate over each row in the DataFrame
for start_index in range(0, len(foodrecipescleaned_df), batch_size):
    end_index = min(start_index + batch_size, len(foodrecipescleaned_df))
    batch_df = foodrecipescleaned_df.iloc[start_index:end_index].copy()
    print('Starting batch size:', batch_df.size)
    for index, row in batch_df.iterrows():
        # Extract the 'titlename' value from the current row
        if isinstance(row['Cleaned_title'], str):
            titlename = row['Cleaned_title'].replace('"', "").replace(":", "").replace("\n", "").replace("-", " ")
        else:
            titlename = str(row['Cleaned_title']).replace('"', "").replace(":", "").replace("\n", "").replace("-", " ")  # Convert to string
        imagename = row['Image_Name']
        # Construct the current and new file paths
        current_file_path = os.path.join(img_path, f"{imagename}.jpg")
        new_file_path = os.path.join(img_path, f"{titlename}.jpg")  # Example: Renaming based on row index
        # Replace the Image_Name also with the Title name so we can fine tune with the same name
        row['Image_Name'] = titlename
        row['Cleaned_title'] = titlename
        
        # Rename the file
        try:
            os.rename(current_file_path, new_file_path)
        except AttributeError as e:
#             print(f"AttributeError occurred: {e}")
#             traceback.print_exc()     
            aeerror_N += 1
            batch_df.drop(index, inplace=True)
            continue
        except OSError as o:
            print(f"OS Error occurred: {o}")
            oserror_N += 1
            batch_df.drop(index, inplace=True)
            continue          
        except FileExistsError:
#             new_file_path = os.path.join(img_path, f"{titlename+str(index)}.jpg")
#             os.rename(current_file_path, new_file_path)
            feerror_N += 1
#             traceback.print_exc()  
            batch_df.drop(index, inplace=True)
            continue
        except FileNotFoundError:
#             print(f"File not found: {current_file_path}. Skipping...")
            fnferror_N += 1
#             traceback.print_exc()
            batch_df.drop(index, inplace=True)
            continue
    print('Ending batch size:', batch_df.size)
    # Concatenate batch_df to merged_df
    foodrecipescleaned_2_df = pd.concat([foodrecipescleaned_2_df, batch_df], ignore_index=True)

# Update the Image_Name column in the DataFrame with the new title names
# foodrecipescleaned_df['Image_Name'] = foodrecipescleaned_df['Cleaned_title']

# Total number of errors
print(f'OS Error {oserror_N}')
print(f'Attribute Error {aeerror_N}')
print(f'FileExists Error {feerror_N}')
print(f'FileNotFound Error {fnferror_N}')
foodrecipescleaned_2_df.head(5)

Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch size: 600
Starting batch size: 600
Ending batch si

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Cleaned_title
0,Miso-Butter Roast Chicken With Acorn Squash Panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more', '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage', '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature', '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper', '⅓ loaf good-quality sturdy white bread, torn into 1"" pieces (about 2½ cups)', '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1"" pieces', '2 Tbsp. extra-virgin olive oil', '½ small red onion, thinly sliced', '3 Tbsp. apple cider vinegar', '1 Tbsp. white miso', '¼ cup all-purpose flour', '2 Tbsp. unsalted butter, room temperature', '¼ cup dry white wine', '2 cups unsalted chicken broth', '2 tsp. white miso', 'Kosher salt, freshly ground pepper']","Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.\nMeanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½""-thick wedges; arrange on a rimmed baking sheet.\nCombine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.\nAdd bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.\nPlace onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.\nPlace a rack in middle and lower third of oven; preheat to 425°F. Mix miso and 3 Tbsp. room-temperature butter in a small bowl until smooth. Pat chicken dry with paper towels, then rub or brush all over with miso butter. Place chicken in a large cast-iron skillet and roast on middle rack until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. (Temperature will climb to 165°F while chicken rests.) Let chicken rest in skillet at least 5 minutes, then transfer to a plate; reserve skillet.\nMeanwhile, roast squash on lower rack until mostly tender, about 25 minutes. Remove from oven and scatter reserved bread mixture over, spreading into as even a layer as you can manage. Return to oven and roast until bread is golden brown and crisp and apples are tender, about 15 minutes. Remove from oven, drain pickled onions, and toss to combine. Transfer to a serving dish.\nUsing your fingers, mash flour and butter in a small bowl to combine.\nSet reserved skillet with chicken drippings over medium heat. You should have about ¼ cup, but a little over or under is all good. (If you have significantly more, drain off and set excess aside.) Add wine and cook, stirring often and scraping up any browned bits with a wooden spoon, until bits are loosened and wine is reduced by about half (you should be able to smell the wine), about 2 minutes. Add butter mixture; cook, stirring often, until a smooth paste forms, about 2 minutes. Add broth and any reserved drippings and cook, stirring constantly, until combined and thickened, 6–8 minutes. Remove from heat and stir in miso. Taste and season with salt and black pepper.\nServe chicken with gravy and squash panzanella alongside.",Miso Butter Roast Chicken With Acorn Squash Panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more', '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage', '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature', '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper', '⅓ loaf good-quality sturdy white bread, torn into 1"" pieces (about 2½ cup

In [43]:
foodrecipescleaned_2_df.head(5)

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Cleaned_title
0,Miso-Butter Roast Chicken With Acorn Squash Panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more', '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage', '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature', '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper', '⅓ loaf good-quality sturdy white bread, torn into 1"" pieces (about 2½ cups)', '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1"" pieces', '2 Tbsp. extra-virgin olive oil', '½ small red onion, thinly sliced', '3 Tbsp. apple cider vinegar', '1 Tbsp. white miso', '¼ cup all-purpose flour', '2 Tbsp. unsalted butter, room temperature', '¼ cup dry white wine', '2 cups unsalted chicken broth', '2 tsp. white miso', 'Kosher salt, freshly ground pepper']","Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.\nMeanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½""-thick wedges; arrange on a rimmed baking sheet.\nCombine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.\nAdd bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.\nPlace onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.\nPlace a rack in middle and lower third of oven; preheat to 425°F. Mix miso and 3 Tbsp. room-temperature butter in a small bowl until smooth. Pat chicken dry with paper towels, then rub or brush all over with miso butter. Place chicken in a large cast-iron skillet and roast on middle rack until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. (Temperature will climb to 165°F while chicken rests.) Let chicken rest in skillet at least 5 minutes, then transfer to a plate; reserve skillet.\nMeanwhile, roast squash on lower rack until mostly tender, about 25 minutes. Remove from oven and scatter reserved bread mixture over, spreading into as even a layer as you can manage. Return to oven and roast until bread is golden brown and crisp and apples are tender, about 15 minutes. Remove from oven, drain pickled onions, and toss to combine. Transfer to a serving dish.\nUsing your fingers, mash flour and butter in a small bowl to combine.\nSet reserved skillet with chicken drippings over medium heat. You should have about ¼ cup, but a little over or under is all good. (If you have significantly more, drain off and set excess aside.) Add wine and cook, stirring often and scraping up any browned bits with a wooden spoon, until bits are loosened and wine is reduced by about half (you should be able to smell the wine), about 2 minutes. Add butter mixture; cook, stirring often, until a smooth paste forms, about 2 minutes. Add broth and any reserved drippings and cook, stirring constantly, until combined and thickened, 6–8 minutes. Remove from heat and stir in miso. Taste and season with salt and black pepper.\nServe chicken with gravy and squash panzanella alongside.",Miso Butter Roast Chicken With Acorn Squash Panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more', '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage', '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature', '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper', '⅓ loaf good-quality sturdy white bread, torn into 1"" pieces (about 2½ cup

In [44]:
foodrecipescleaned_2_df.loc[foodrecipescleaned_2_df['Image_Name']=="crab-sweet-corn-cakes-365969"]

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Cleaned_title
5290,Crab & Sweet Corn Cakes,"['8 ounces cooked crabmeat', '2 green onions, thinly sliced', '1 red serrano chile, finely chopped', '1 tablespoon roughly chopped fresh cilantro leaves', '1/2 cup all-purpose flour', '1/2 cup cornstarch', '2 eggs, beaten', '1/2 cup canned corn kernels, drained', '2/3 cup crème fraîche or sour cream, to serve', 'sweet chile sauce, to serve (see note)', 'lime wedges, to serve']","Lightly mix the crab, green onions, chile and cilantro in a bowl and season with salt and freshly ground black pepper.\nSift the flour and cornstarch into a separate bowl. Add 2/3 cup cold water and the eggs and whisk until smooth. Stir in the crab mixture and corn kernels. The mixture should have the consistency of heavy cream.\nSet up the outdoor grill for direct-heat cooking over medium-high heat. Place a griddle, plancha or piastra suitable for use on an outdoor grill on the grill to preheat. Grease the griddle with some oil. Spoon small amounts of the batter onto the griddle and cook for 3 minutes on each side or until golden. Serve with crème fraîche, sweet chile sauce and lime wedges.",crab-sweet-corn-cakes-365969,"['8 ounces cooked crabmeat', '2 green onions, thinly sliced', '1 red serrano chile, finely chopped', '1 tablespoon roughly chopped fresh cilantro leaves', '1/2 cup all-purpose flour', '1/2 cup cornstarch', '2 eggs, beaten', '1/2 cup canned corn kernels, drained', '2/3 cup crème fraîche or sour cream, to serve', 'sweet chile sauce, to serve (see note)', 'lime wedges', 'to serve']",Crab & Sweet Corn Cakes


### Test and Val split

In [45]:
features_columns = ['Cleaned_title', 'Instructions', 'Cleaned_Ingredients']

label_columns = 'Image_Name'
df, train_df, val_df = create_train_test_split(foodrecipescleaned_2_df, features_columns, label_columns)

In [46]:
train_df.shape

(7752, 4)

In [47]:
train_df.head()

,Cleaned_title,Instructions,Cleaned_Ingredients,Image_Name
0,Pot Roast in Rich Gravy,"Preheat oven to 350°F with rack in middle.\nOn a large plate mix together matzoh cake meal with 1 tablespoon kosher salt and 1/2 teaspoon pepper. Pat meat dry then coat with matzoh meal mixture, dusting off excess (discard matzoh meal but reserve plate).\nHeat 3 tablespoons oil in a 6- to 7-quart-wide heavy pot over medium high heat until oil shimmers. Brown meat on all sides, totaling about 12 minutes. Transfer to plate.\nReduce heat to medium and add onions, carrots, and parsnips to pot with 1 teaspoon kosher salt and 1/2 teaspoon pepper and cook, stirring occasionally, until vegetables begin to soften and turn golden, 8 to 10 minutes.\nAdd wine to pot and boil, scraping up any bits from bottom of pan until wine is reduced by half.\nAdd chicken broth, tomatoes in juice, thyme, bay leaves, and cinnamon stick to pot and bring to a simmer. Return meat to pot and cover with lid. Place in oven and braise until meat is tender when pierced with a fork, about 3 hours.\nTransfer meat to a cutting board. Remove thyme sprigs, bay leaves, and cinnamon stick. Transfer 2 cups braising liquid to a blender and purée until smooth (use caution when pureeing hot liquids). Stir pureed sauce into liquid in pot and adjust seasoning if necessary.\nDiscard strings from meat and cut into thick slices (it may fall apart as you slice it because it's so tender), then return meat to sauce and serve.","['1/4 cup matzoh cake meal', '4 tablespoons vegetable or olive oil, divided', '1 (4 1/2-to 5-pound) beef chuck roast, tied', '1 large onion (about 1 pound), coarsely chopped (2 cups)', '2 large carrots (about 1/2 pound), coarsely chopped', '2 medium parsnips (about 1/2 pound), coarsely chopped', '2 cups full-bodied red wine', '4 cups chicken broth', '1 (28-ounce) can diced tomatoes in juice', '5 full sprigs fresh thyme', '2 bay leaves', '1 (3-inch long) cinnamon stick', 'Fresh horseradish root, peeled and finely grated for serving (optional)', 'a 6- to 7-quart-wide heavy pot with lid']",Pot Roast in Rich Gravy
1,Elvis Pie,"Make the crust: Preheat the oven to 350°F (175°C). On a foil-lined baking sheet, cook the bacon until it is super crispy, but not burnt. Remove it from the oven (keep the oven on), and drain the bacon on paper towels. When the bacon is cool enough to handle, process it in a food processor with the cookies until you've got fine crumbs. Mix the crumbs with the fat until the crumbs are the consistency of wet sand. Press them firmly into a 9-inch (23-cm) pie plate and chill the crust until firm. Bake the crust for about 10 minutes. Allow it to cool completely.\nMake the filling: Prepare an ice bath. In a small bowl, whisk the egg yolks lightly. In a large saucepan over medium heat, whisk together the milk, granulated sugar, cornstarch, and salt. Bring the mixture to a simmer but do not let it boil, whisking constantly for 4 minutes.\nTemper the egg yolks by whisking them constantly as you ladle in a thin stream of the hot milk mixture. Add another ladleful of milk, whisking constantly, then pour the egg mixture back into the saucepan of hot milk as you whisk. Cook it over medium-high heat, whisking steadily, until the custard is thick and you see small bubbles breaking the surface, about 3 minutes. Remove it from the heat, whisk in the vanilla, and set the pan in the ice bath. Stir it occasionally with a spatula until fully cooled. (This can be made ahead and refrigerated for up to 1 day, covered tightly with plastic wrap. Make sure the plastic touches the surface of the custard, to prevent a skin from forming.)\nTo assemble the pie, slice the bananas lengthwise or in circles, whichever you prefer, and arrange a layer of them in the bottom of your pie crust. Cover them with a layer of pudding, then repeat with the remaining banana slices and pudding.\nMake the topping: In a stand mixer, with a hand mixer, or by hand with a whisk, whip the cream, powdered su

In [48]:
val_df.shape

(2584, 4)

## Prepping the dataset for image generation

### Splitting the string in the Instructions and Cleaned_ingredients columns into sentences and storing it in one column to enable easier looping and generation for images.

### Combining split sentences of ingredients and instructions into one column by title.

In [49]:
# import ast
# import json

# import spacy

# # Load the English language model in spacy
# nlp = spacy.load('en_core_web_sm')

# # Define batch size
# batch_size = 100

def split_instructions_and_ingredients_into_sentences(df):
    # Initialize an empty list to store DataFrames for each batch
    batch_dfs = []

    # Process rows in batches
    for start_index in range(0, len(df), batch_size):
        end_index = min(start_index + batch_size, len(df))
        batch_df = df.iloc[start_index:end_index].copy()  # Make a copy of the batch DataFrame
        # Drop unnecessary columns: Ingredients
#         batch_df.drop(columns=['Ingredients'], inplace=True)
    
        # Tokenize sentences in the 'Instructions' column
        batch_df['Instructions'] = batch_df['Instructions'].astype(str).apply(lambda x: [sent.text.strip() for sent in nlp(x).sents])
   
        # Convert string representation of list to list
        batch_df['Cleaned_Ingredients'] = batch_df['Cleaned_Ingredients'].apply(eval)
        
        batch_df['image_path'] = img_path + "\\" + df['Cleaned_title'] + ".jpg"  #was Image_Name, changed to Clean_title
        print('image_path: ', batch_df['image_path'])
   
        # Separate cleaned ingredients and instructions into separate DataFrames
        instructions_df = batch_df.explode('Instructions')[['Cleaned_title', 'Instructions', 'Image_Name', 'image_path']].copy()
        cleaned_ingredients_df = batch_df.explode('Cleaned_Ingredients')[['Cleaned_title', 'Cleaned_Ingredients', 'Image_Name', 'image_path']].copy()

        # Rename the columns for concatenation
        instructions_df.columns = ['Title', 'Recipe', 'Image_Name', 'image_path']
        cleaned_ingredients_df.columns = ['Title', 'Recipe', 'Image_Name', 'image_path']

        # Add a column to differentiate between cleaned ingredients and instructions
        instructions_df['Type'] = 'Instructions'
        cleaned_ingredients_df['Type'] = 'Cleaned Ingredients'
    
        # Concatenate the DataFrames and append to the list
        batch_dfs.append(pd.concat([cleaned_ingredients_df, instructions_df], ignore_index=True))


    # Concatenate the DataFrames vertically
    return_df = pd.concat(batch_dfs, ignore_index=True)
    return return_df

# # Print the resulting DataFrame
# print(combined_df)
# # Print total number of rows processed
# print("Total number of rows processed:", len(combined_df))


### Drop rows from training and test that have no instructions

#### Training data

In [50]:
train_df.loc[train_df['Image_Name']=="Broccolini Cheddar Gratin with Rye Breadcrumbs"]
# The ingredients are there, but instructions aren't. Drop it.

,Cleaned_title,Instructions,Cleaned_Ingredients,Image_Name
1430,Broccolini Cheddar Gratin with Rye Breadcrumbs,,"['1/4 loaf seeded rye bread, torn into 1"" pieces (about 2 cups)', '4 dried chiles de árbol, divided', '4 tablespoons olive oil, divided', 'Kosher salt', 'freshly ground pepper', '1 pound broccolini, larger stalks halved lengthwise', '4 tablespoons finely grated Parmesan, divided', '4 ounces white cheddar', 'thinly sliced']",Broccolini Cheddar Gratin with Rye Breadcrumbs


In [51]:
#  Drop rows with blank instructions
indices_of_rows_to_drop = train_df[train_df['Instructions'].str.strip().eq('')].index
print(indices_of_rows_to_drop)
train_df = train_df.drop(indices_of_rows_to_drop)

Int64Index([1430, 1664, 1902, 5606, 5973], dtype='int64')


In [52]:
train_df.loc[train_df['Image_Name']=="_12829"]

,Cleaned_title,Instructions,Cleaned_Ingredients,Image_Name


In [53]:
train_df.columns, train_df.shape

(Index(['Cleaned_title', 'Instructions', 'Cleaned_Ingredients', 'Image_Name'], dtype='object'),
 (7747, 4))

In [54]:
train_df_split = split_instructions_and_ingredients_into_sentences(train_df)

image_path:  0                                    Food Recipes\Food Images\Pot Roast in Rich Gravy.jpg
1                                                  Food Recipes\Food Images\Elvis Pie.jpg
2                           Food Recipes\Food Images\Roast Pumpkin with Cheese Fondue.jpg
3                                 Food Recipes\Food Images\Chocolate Cherry Cola Cake.jpg
4                          Food Recipes\Food Images\Spiced Cucumbers and Coconut Milk.jpg
                                             ...                                         
95                                   Food Recipes\Food Images\Sweet Preserved Pumpkin.jpg
96                       Food Recipes\Food Images\Mango, Mint, and Pineapple Smoothie.jpg
97                        Food Recipes\Food Images\Flaxseed, Fig, and Walnut Crackers.jpg
98                             Food Recipes\Food Images\Chickpea and Chorizo Tostadas.jpg
99    Food Recipes\Food Images\Linguine Al Limone with Grilled Chia Chicken Meatballs.j

image_path:  800                       Food Recipes\Food Images\Pressed Broccoli Rabe and Mozzarella Sandwiches.jpg
801                                                   Food Recipes\Food Images\Green Tea Margarita.jpg
802                                                  Food Recipes\Food Images\Pineapple Mint Vodka.jpg
803                                                               Food Recipes\Food Images\Dry Rub.jpg
804                                    Food Recipes\Food Images\Giant Meringue (Pavlova Gigantata).jpg
                                                    ...                                               
895                                   Food Recipes\Food Images\Sautéed Greens with Toasted Walnuts.jpg
896           Food Recipes\Food Images\Raspberry Cake with Marsala, Crème Fraîche, and Raspberries.jpg
897    Food Recipes\Food Images\Rib Eye Steaks with Gorgonzola Butter and Crispy Sweet Onion Rings.jpg
898                                          Food Recipes\Fo

image_path:  1601    Food Recipes\Food Images\Crab Hush Puppies With Curried Honey Mustard Sauce.jpg
1602         Food Recipes\Food Images\Roasted Cauliflower with Cheddar Cheese Sauce.jpg
1603                                  Food Recipes\Food Images\Cauliflower Sformati.jpg
1604                                          Food Recipes\Food Images\Yaki Onigiri.jpg
1605                    Food Recipes\Food Images\Brocollini with Pecan Brown Butter.jpg
                                             ...                                       
1697     Food Recipes\Food Images\Breakfast Grilled Cheese with Soft Scrambled Eggs.jpg
1698                      Food Recipes\Food Images\Egg Drop Soup with Fresh Spinach.jpg
1699            Food Recipes\Food Images\Hardy Greens With Lemon Garlic Vinaigrette.jpg
1700                                               Food Recipes\Food Images\Tuckers.jpg
1701                  Food Recipes\Food Images\Eggplant and Sun dried Tomato Spread.jpg
Name: image_path, L

image_path:  2403             Food Recipes\Food Images\Braised Chicken with Celery Root and Garlic.jpg
2404                              Food Recipes\Food Images\The Garlickiest Fried Rice.jpg
2405                        Food Recipes\Food Images\Grilled Branzino with Ladolemono.jpg
2406                                     Food Recipes\Food Images\Shaved Fennel Salad.jpg
2407         Food Recipes\Food Images\Hot Smoked Salmon with Salted Yogurt and Fennel.jpg
                                              ...                                        
2498     Food Recipes\Food Images\Steak with Watercress Salad and Chile Lime Dressing.jpg
2499                                  Food Recipes\Food Images\Black Pearl Layer Cake.jpg
2500    Food Recipes\Food Images\Mashed Yukon Gold Potatoes with Caramelized Shallots.jpg
2501                     Food Recipes\Food Images\Turkey for Two with Pan Sauce Gravy.jpg
2502                           Food Recipes\Food Images\Chipotle Lime Grilled Chicken.j

image_path:  3203                                     Food Recipes\Food Images\Pecan Praline Trellis.jpg
3204            Food Recipes\Food Images\Agave Glazed Pork Belly with Grilled Pineapple.jpg
3205         Food Recipes\Food Images\Apple Cranberry Crisp with Oatmeal Cookie Crumble.jpg
3206                                        Food Recipes\Food Images\Ranch Bison Burger.jpg
3207                         Food Recipes\Food Images\Sesame Seed and Árbol Chile Salsa.jpg
                                               ...                                         
3298    Food Recipes\Food Images\Sausage, Roasted Red Pepper, and Spinach Torta Rustica.jpg
3299                                Food Recipes\Food Images\Lamb with Preserved Lemons.jpg
3300                                     Food Recipes\Food Images\Buttery Sugar Cookies.jpg
3301      Food Recipes\Food Images\Smoked Turkey, Blue Cheese, and Red Onion Sandwiches.jpg
3302                             Food Recipes\Food Images\Tomato, F

...

image_path:  4703                               Food Recipes\Food Images\Green Chili with Pork.jpg
4704                                  Food Recipes\Food Images\Apples and Oranges.jpg
4705                            Food Recipes\Food Images\Pan Seared Carrot Steaks.jpg
4706               Food Recipes\Food Images\Tandoori Style Grilled Meat or Shrimp.jpg
4707      Food Recipes\Food Images\Crushed Cucumbers with Lime Pickle and Coconut.jpg
                                            ...                                      
4798               Food Recipes\Food Images\Chicken Soup with Stars and Meatballs.jpg
4799    Food Recipes\Food Images\Dark Chocolate and Peppermint Whipped Cream Tart.jpg
4800                            Food Recipes\Food Images\Fresh Coconut Layer Cake.jpg
4801    Food Recipes\Food Images\Lobster Toasts With Avocado and Espelette Pepper.jpg
4802                      Food Recipes\Food Images\Salted Black Licorice Caramels.jpg
Name: image_path, Length: 100, dtype: obj

image_path:  5503                                     Food Recipes\Food Images\Sangria Blanco_4554.jpg
5504                           Food Recipes\Food Images\Peach Blueberry Ice Cream Pie.jpg
5505    Food Recipes\Food Images\Confetti Corn BreadCrusted Shrimp in Creole Filling.jpg
5506                   Food Recipes\Food Images\Fresh Orange Tart with Hazelnut Crust.jpg
5507                              Food Recipes\Food Images\Quince and Currant Chutney.jpg
                                              ...                                        
5598            Food Recipes\Food Images\Fingerling Potatoes with Chives and Tarragon.jpg
5599                              Food Recipes\Food Images\Thai Green Curry Paste_332.jpg
5600                           Food Recipes\Food Images\Grilled Marinated Leg of Lamb.jpg
5601                           Food Recipes\Food Images\Melon with Basil Lime Granita.jpg
5602               Food Recipes\Food Images\Rock Shrimp Pasta with Spicy Tomato Sauce.j

image_path:  6305      Food Recipes\Food Images\Roast Lamb with Lamb Sausage Crust and Fresh Grape Pan Sauce.jpg
6306            Food Recipes\Food Images\Stacked Chicken Enchiladas with Salsa Verde and Cheese.jpg
6307                 Food Recipes\Food Images\Grilled Halibut with Tatsoi and Spicy Thai Chiles.jpg
6308                                                      Food Recipes\Food Images\Brothy Beans.jpg
6309                                            Food Recipes\Food Images\Pulled Brisket Sliders.jpg
                                                   ...                                             
6400    Food Recipes\Food Images\Roasted Oysters with Pickled Radishes, Carrots and Celery Root.jpg
6401                                 Food Recipes\Food Images\Roasted Halibut with Garlic Sauce.jpg
6402                        Food Recipes\Food Images\Shredded Brussels Sprout and Ricotta Toast.jpg
6403                                               Food Recipes\Food Images\Chicken Fri

image_path:  7005                           Food Recipes\Food Images\Beet and Fennel Soup with Kefir.jpg
7006                                   Food Recipes\Food Images\Quick and Easy Cioppino.jpg
7007                Food Recipes\Food Images\Rice Vermicelli with Chicken and Nuoc Cham.jpg
7008                               Food Recipes\Food Images\Turkey and Apricot Meatloaf.jpg
7009                                     Food Recipes\Food Images\Radishes with Burrata.jpg
                                               ...                                         
7100                 Food Recipes\Food Images\Eggs with Cream, Spinach, and Country Ham.jpg
7101                                  Food Recipes\Food Images\Artichoke and Feta Tarts.jpg
7102            Food Recipes\Food Images\Marinated Beans with Celery and Ricotta Salata.jpg
7103                                        Food Recipes\Food Images\Pink Lemonade Bars.jpg
7104    Food Recipes\Food Images\Scallops with Tarragon Cream and W

image_path:  7705                                                         Food Recipes\Food Images\Crispy Pork Lettuce Wraps With Spicy Cucumbers.jpg
7706                                                                   Food Recipes\Food Images\Grilled Citrus Chicken Under a Brick.jpg
7707                                                  Food Recipes\Food Images\Castellane with Mascarpone and Roasted Grape Tomatoes.jpg
7708                                                                            Food Recipes\Food Images\Celery Root and Pecan Salad.jpg
7709                                                                                Food Recipes\Food Images\Basic Hollandaise Sauce.jpg
7710                                                           Food Recipes\Food Images\Sumac Skirt Steak with Pomegranate Reduction.jpg
7711                                                                     Food Recipes\Food Images\Pistachio and Dried Fruit Haroseth.jpg
7712                        

In [55]:
train_df_split.loc[train_df_split['Title']=='Mango, Mint, and Pineapple Smoothie']

,Title,Recipe,Image_Name,image_path,Type
1026,"Mango, Mint, and Pineapple Smoothie",1/2 cup chopped mango,"Mango, Mint, and Pineapple Smoothie","Food Recipes\Food Images\Mango, Mint, and Pineapple Smoothie.jpg",Cleaned Ingredients
1027,"Mango, Mint, and Pineapple Smoothie",1/2 cup chopped pineapple,"Mango, Mint, and Pineapple Smoothie","Food Recipes\Food Images\Mango, Mint, and Pineapple Smoothie.jpg",Cleaned Ingredients
1028,"Mango, Mint, and Pineapple Smoothie",1/2 cup nonfat high-protein plain Greek yogurt or kefir,"Mango, Mint, and Pineapple Smoothie","Food Recipes\Food Images\Mango, Mint, and Pineapple Smoothie.jpg",Cleaned Ingredients
1029,"Mango, Mint, and Pineapple Smoothie",4 mint leaves,"Mango, Mint, and Pineapple Smoothie","Food Recipes\Food Images\Mango, Mint, and Pineapple Smoothie.jpg",Cleaned Ingredients
1030,"Mango, Mint, and Pineapple Smoothie",torn into pieces,"Mango, Mint, and Pineapple Smoothie","Food Recipes\Food Images\Mango, Mint, and Pineapple Smoothie.jpg",Cleaned Ingredients
2399,"Mango, Mint, and Pineapple Smoothie",Place all the ingredients and 3 ice cubes in a blender and blend on high speed for about 60 seconds or until smooth.,"Mango, Mint, and Pineapple Smoothie","Food Recipes\Food Images\Mango, Mint, and Pineapple Smoothie.jpg",Instructions
2400,"Mango, Mint, and Pineapple Smoothie",Pour into a chilled glass and enjoy.,"Mango, Mint, and Pineapple Smoothie","Food Recipes\Food Images\Mango, Mint, and Pineapple Smoothie.jpg",Instructions


#### Test/Val

In [56]:
indices_of_rows_to_drop_val = val_df[val_df['Instructions'].str.strip().eq('')].index
val_df.drop(indices_of_rows_to_drop_val, inplace=True)

In [57]:
val_df.shape

(2582, 4)

In [58]:
val_df_split = split_instructions_and_ingredients_into_sentences(val_df)

image_path:  0              Food Recipes\Food Images\African Curried Coconut Soup with Chickpeas.jpg
1               Food Recipes\Food Images\Lamb Köfte with Yogurt Sauce and Muhammara.jpg
2             Food Recipes\Food Images\Seared Halloumi with Peanut Dukkah and Honey.jpg
3             Food Recipes\Food Images\Smoked Salmon Toasts with Malt Vinegar Syrup.jpg
4                              Food Recipes\Food Images\Kimchi and Miso Noodle Soup.jpg
                                            ...                                        
95    Food Recipes\Food Images\Orange Poppy Seed Pancakes with Rosemary Maple Syrup.jpg
96                       Food Recipes\Food Images\German Style Potato and Ham Salad.jpg
97         Food Recipes\Food Images\Poached Eggs with Mushroom, Tamarillo, and Sage.jpg
98                     Food Recipes\Food Images\Lemon Raspberry French Toast Strata.jpg
99            Food Recipes\Food Images\Apricot Linzertorte with Quark Whipped Cream.jpg
Name: image_path, L

image_path:  801                                               Food Recipes\Food Images\Very Berry Crumble.jpg
802            Food Recipes\Food Images\Charred Sweet Potatoes With Hot Honey Butter and Lime.jpg
803    Food Recipes\Food Images\Cider Dijon Pork Chops with Roasted Sweet Potatoes and Apples.jpg
804                                                Food Recipes\Food Images\Coconut Cream Pie.jpg
805                               Food Recipes\Food Images\Roasted Beet and Goat Cheese Salad.jpg
                                                  ...                                            
896                    Food Recipes\Food Images\Pressure Cooker Mini Meatballs With Radiatori.jpg
897                                            Food Recipes\Food Images\Cucumber Apple Pickle.jpg
898                                 Food Recipes\Food Images\Yogurt Mousse with Apricot Sauce.jpg
899                             Food Recipes\Food Images\3 Ingredient Microwave Caramel Sauce.jpg
900    

image_path:  1601                                    Food Recipes\Food Images\Raspberry Jam Tart with Almond Crumble.jpg
1602                         Food Recipes\Food Images\Grilled Beef Wrapped in Sesame Leaves (Bo La Lot).jpg
1603                                      Food Recipes\Food Images\Potato Salad with Olives and Peppers.jpg
1604                                    Food Recipes\Food Images\Hush Up Spicy Gluten Free Hush Puppies.jpg
1605                          Food Recipes\Food Images\Ginger Scallion Meatballs With Green Curry Sauce.jpg
                                                       ...                                                 
1696                                                   Food Recipes\Food Images\Chicken Wings Five Ways.jpg
1697         Food Recipes\Food Images\Inside Out Grilled Cheese with Fig Chutney and Caramelized Onions.jpg
1698                                                    Food Recipes\Food Images\Corn and Tomato Gratin.jpg
1699    Food Re

image_path:  2402                     Food Recipes\Food Images\Tomato Watermelon Salad with Turmeric Oil.jpg
2403                                    Food Recipes\Food Images\Shrimp and Corn with Basil.jpg
2404                Food Recipes\Food Images\Seared Hake with Baby Potatoes and Green Sauce.jpg
2405    Food Recipes\Food Images\Avocado Soup with Herbs, Slivered Radishes, and Pistachios.jpg
2406                       Food Recipes\Food Images\Guanabana Sorbet with Mango Lime Coulis.jpg
                                                 ...                                           
2497                                  Food Recipes\Food Images\Sumac, Spelt, and Apple Cake.jpg
2498                                   Food Recipes\Food Images\Lemon Cranberry Florentines.jpg
2499                                   Food Recipes\Food Images\Salted Honey Cantaloupe Jam.jpg
2500                                                     Food Recipes\Food Images\Lamb Larb.jpg
2501                       

In [59]:
val_df_split

,Title,Recipe,Image_Name,image_path,Type
0,African Curried Coconut Soup with Chickpeas,2 tablespoons canola oil,African Curried Coconut Soup with Chickpeas,Food Recipes\Food Images\African Curried Coconut Soup with Chickpeas.jpg,Cleaned Ingredients
1,African Curried Coconut Soup with Chickpeas,"1 medium onion (about 6 ounces), chopped",African Curried Coconut Soup with Chickpeas,Food Recipes\Food Images\African Curried Coconut Soup with Chickpeas.jpg,Cleaned Ingredients
2,African Curried Coconut Soup with Chickpeas,"1 medium red bell pepper (about 6 ounces), chopped",African Curried Coconut Soup with Chickpeas,Food Recipes\Food Images\African Curried Coconut Soup with Chickpeas.jpg,Cleaned Ingredients
3,African Curried Coconut Soup with Chickpeas,"1 jalapeño chili, seeded and finely chopped",African Curried Coconut Soup with Chickpeas,Food Recipes\Food Images\African Curried Coconut Soup with Chickpeas.jpg,Cleaned Ingredients
4,African Curried Coconut Soup with Chickpeas,"2 large cloves garlic, finely chopped",African Curried Coconut Soup with Chickpeas,Food Recipes\Food Images\African Curried Coconut Soup with Chickpeas.jpg,Cleaned Ingredients
...,...,...,...,...,...
67646,Roast Sirloin of Beef,"Transfer skillet to oven and roast until an instant-read thermometer inserted into center of roast registers 120°F for medium-rare, 40-50 minutes.",Roast Sirloin of Beef,Food Recipes\Food Images\Roast Sirloin of Beef.jpg,Instructions
67647,Roast Sirloin of Beef,Tent with foil; let rest 15 minutes.,Roast Sirloin of Beef,Food Recipes\Food Images\Roast Sirloin of Beef.jpg,Instructions
67648,Roast Sirloin of Beef,Thinly slice roast.,Roast Sirloin of Beef,Food Recipes\Food Images\Roast Sirloin of Beef.jpg,Instructions
67649,Roast Sirloin of Beef,Arrange on a platter and sprinkle with additional gray sea salt.,Roast Sirloin of Beef,Food Recipes\Food Images\Roast Sirloin of Beef.jpg,Instructions


In [60]:
val_df_split.loc[val_df_split['Image_Name']=="Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries"]

,Title,Recipe,Image_Name,image_path,Type
45201,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries",1/2 cup dried tart cherries,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries","Food Recipes\Food Images\Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries.jpg",Cleaned Ingredients
45202,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries",1 (1-pound) loaf crusty country-style white bread,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries","Food Recipes\Food Images\Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries.jpg",Cleaned Ingredients
45203,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries",1/4 cup olive oil,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries","Food Recipes\Food Images\Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries.jpg",Cleaned Ingredients
45204,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries",4 teaspoons chopped fresh thyme,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries","Food Recipes\Food Images\Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries.jpg",Cleaned Ingredients
45205,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries","1 large garlic clove, minced","Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries","Food Recipes\Food Images\Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries.jpg",Cleaned Ingredients
45206,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries",6 tablespoons (3/4 stick) butter,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries","Food Recipes\Food Images\Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries.jpg",Cleaned Ingredients
45207,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries",1 1/2 cups finely chopped onion,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries","Food Recipes\Food Images\Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries.jpg",Cleaned Ingredients
45208,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries",1 1/2 cups thinly sliced celery,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries","Food Recipes\Food Images\Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries.jpg",Cleaned Ingredients
45209,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries",1 1/4 cups chopped smoked ham (preferably from 2 meaty ham hocks),"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries","Food Recipes\Food Images\Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries.jpg",Cleaned Ingredients
45210,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries",1 cup finely chopped green bell pepper,"Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries","Food Recipes\Food Images\Country Bread Stuffing with Smoked Ham, Goat Cheese, and Dried Cherries.jpg",Cleaned Ingredients


In [61]:
import os
import shutil
import tempfile
import traceback

batch_size = 100
fnferror_zip_N = 0
oserror_zip_N = 0

def zipit(output_dir, df):
    global fnferror_zip_N, oserror_zip_N
    os.makedirs(output_dir, exist_ok=True)

    # Create a temporary directory to store all images
    temp_dir = tempfile.mkdtemp()

    # Group image paths by title
    grouped_paths = df.groupby('Title')['image_path'].apply(list)

    # Iterate over each group
    for title, image_paths in grouped_paths.items():
        # Write batch data to individual files
        for i, file_path in enumerate(image_paths):
            try:
                normalized_file_path = os.path.normpath(file_path)
                with open(os.path.join(temp_dir, f"{title}.jpg"), "wb") as file:
                    try:
                        with open(normalized_file_path, "rb") as image_file:
                            file.write(image_file.read())  # Write data to file
                    except FileNotFoundError:
                        continue 
                    except OSError:
                        continue 
            except FileNotFoundError:
#                 print(f"File not found: {normalized_file_path}. Skipping...")
                fnferror_zip_N += 1
#                 traceback.print_exc()
                continue
            except OSError:
                oserror_zip_N += 1
#                 traceback.print_exc()
                continue
    # Create a ZIP archive for the batch
    shutil.make_archive(os.path.join(output_dir, "batch_images"), "zip", temp_dir)

    # Remove temporary files after creating ZIP archives
    shutil.rmtree(temp_dir)


In [62]:
# Drop duplicates based on 'Title' and 'image_path' columns, keeping the first occurrence
unique_data_train = train_df_split.drop_duplicates(subset=['Title', 'image_path'], keep='first').reset_index(drop=True)

# Display the resulting DataFrame
# print(unique_data_train)

zipit("Training_data", unique_data_train)

In [63]:
# Already done.
# train_df_split.to_csv('Food Recipes\\food_train.csv', index=False)

In [65]:
# Drop duplicates based on 'Title' and 'image_path' columns, keeping the first occurrence
unique_data_test = val_df_split.drop_duplicates(subset=['Title', 'image_path'], keep='first').reset_index(drop=True)
unique_data_test.head(10)

,Title,Recipe,Image_Name,image_path,Type
0,African Curried Coconut Soup with Chickpeas,2 tablespoons canola oil,African Curried Coconut Soup with Chickpeas,Food Recipes\Food Images\African Curried Coconut Soup with Chickpeas.jpg,Cleaned Ingredients
1,Lamb Köfte with Yogurt Sauce and Muhammara,1 cup plain low-fat yogurt,Lamb Köfte with Yogurt Sauce and Muhammara,Food Recipes\Food Images\Lamb Köfte with Yogurt Sauce and Muhammara.jpg,Cleaned Ingredients
2,Seared Halloumi with Peanut Dukkah and Honey,1/4 cup skinless raw peanuts,Seared Halloumi with Peanut Dukkah and Honey,Food Recipes\Food Images\Seared Halloumi with Peanut Dukkah and Honey.jpg,Cleaned Ingredients
3,Smoked Salmon Toasts with Malt Vinegar Syrup,1/2 cup apple cider,Smoked Salmon Toasts with Malt Vinegar Syrup,Food Recipes\Food Images\Smoked Salmon Toasts with Malt Vinegar Syrup.jpg,Cleaned Ingredients
4,Kimchi and Miso Noodle Soup,"3 cups (200 g ) Asian mushrooms (enoki, shimeji, shiitake, oyster)",Kimchi and Miso Noodle Soup,Food Recipes\Food Images\Kimchi and Miso Noodle Soup.jpg,Cleaned Ingredients
5,Peanut Butter Cheesecake with Caramelized Banana Topping,"18 whole graham crackers, coarsely broken",Peanut Butter Cheesecake with Caramelized Banana Topping,Food Recipes\Food Images\Peanut Butter Cheesecake with Caramelized Banana Topping.jpg,Cleaned Ingredients
6,Spiced Lamb and Dill Yogurt Pasta,3 large egg yolks,Spiced Lamb and Dill Yogurt Pasta,Food Recipes\Food Images\Spiced Lamb and Dill Yogurt Pasta.jpg,Cleaned Ingredients
7,Gribiche (Hard Boiled Egg) Dressing,"6 cornichons, chopped",Gribiche (Hard Boiled Egg) Dressing,Food Recipes\Food Images\Gribiche (Hard Boiled Egg) Dressing.jpg,Cleaned Ingredients
8,"Bucatini with Olives, Sun Dried Tomatoes, and Basil",Kosher salt,"Bucatini with Olives, Sun Dried Tomatoes, and Basil","Food Recipes\Food Images\Bucatini with Olives, Sun Dried Tomatoes, and Basil.jpg",Cleaned Ingredients
9,Mustard Cream Sauce,1/2 cup dry white wine,Mustard Cream Sauce,Food Recipes\Food Images\Mustard Cream Sauce.jpg,Cleaned Ingredients


In [66]:
zipit("Test_data", unique_data_test)

In [67]:
# Already done.
# val_df_split.to_csv('Food Recipes\\food_test.csv', index=False)